In [21]:
#環境設置
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
import time
driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver92") 

def organize_columns(df1):
    # 合併全部的 DataFrame
    try:
        df1 = pd.concat(df1, axis=0, ignore_index=True)
    except:
        df1.reset_index(drop=True, inplace=True)

    # 處理 column 2：館藏地
    c2 = [
        '分館/專室', '館藏地/室', '館藏室', '館藏地/館藏室', '館藏地', '典藏館', '館藏位置', '館藏地/區域',
        '典藏地名稱', '館藏地/館別', '館藏地(已外借/總數)'
    ]
    df1['c2'] = ''
    for c in c2:
        try:
            df1['c2'] += df1[c]
        except:
            pass

    # 處理 column 3：索書號
    c3 = ['索書號', '索書號/期刊合訂本卷期', '索書號 / 部冊號']
    df1['c3'] = ''
    for c in c3:
        try:
            df1['c3'] += df1[c]
        except:
            pass

    # 處理 column 4：館藏狀態
    c4 = [
        '館藏位置(到期日期僅為期限，不代表上架日期)', '狀態/到期日', '目前狀態 / 到期日', '館藏狀態', '處理狀態',
        '狀態 (說明)', '館藏現況 說明', '目前狀態/預計歸還日期', '圖書狀況 / 到期日', '調閱說明', '借閱狀態',
        '狀態', '館藏狀態(月-日-西元年)', '圖書狀況', '現況/異動日', 'Unnamed: 24'
    ]
    df1['c4'] = ''
    for c in c4:
        try:
            df1['c4'] += df1[c]
        except:
            pass

    # 直接生成另一個 DataFrame
    df2 = pd.DataFrame()
    df2['圖書館'] = df1['圖書館']
    df2['館藏地'] = df1['c2']
    df2['索書號'] = df1['c3']
    df2['館藏狀態'] = df1['c4']
    df2['連結'] = df1['連結']

    # 遇到值為 NaN時，將前一列的值填補進來
    df2.fillna(method="ffill", axis=0, inplace=True)

    return df2


def accurately_find_table_and_read_it(table_position, table_position2=0):
    try:
        if not wait_for_element_present(table_position):
            print(f'找不到 {table_position}！')
            return
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        table_innerHTML = soup.select(table_position)[table_position2]
        tgt = pd.read_html(str(table_innerHTML), encoding='utf-8')[0]
        # tgt['圖書館'], tgt['連結'] = org, driver.current_url
    except:
        return
    else:
        return tgt

In [30]:
#台中科技大學
org = "台中科技大學"
ISBN = str(9789574672028)
df_lst = []
url = "https://ntit.ent.sirsidynix.net/client/zh_TW/NUTC/search/results?qu=" + ISBN + "&te=ILS"


driver.get(url)
time.sleep(3)
into_first = driver.find_element_by_id("detailLink0").click() #進第一個版本
time.sleep(5)

print(accurately_find_table_and_read_it("#detailItemsDiv0 > div > table", table_position2=0))


            


None


In [22]:
#台中科技大學
org = "台中科技大學"
ISBN = str(9789574672028)
df_lst = []
url = "https://ntit.ent.sirsidynix.net/client/zh_TW/NUTC/search/results?qu=" + ISBN + "&te=ILS"


driver.get(url)
time.sleep(3)
into_first = driver.find_element_by_id("detailLink0").click() #進第一個版本
time.sleep(3)
for i in range(0, 6):
    booktype = driver.find_element_by_class_name("detailItemsTable_ITYPE").text
    if booktype == "電子資源": #不爬是電子資源的
        pass
    else:
        for i in range(0, 6):
            table_position = "#detailItemsDiv" + str(i) + "> div > table"
            try:
                edition = driver.find_element_by_css_selector(table_position) # 如果那個版本在
                df_lst.append(accurately_find_table_and_read_it(table_position, table_position2=0))
                try:
                    next = driver.find_element_by_class_name("nextArrowRight").click() #按去下一個版本
                except:
                    pass
            except:
                pass
            
print(df_lst)

[None, None, None, None, None, None, None, None]


In [20]:
#台中科技大學
org = "台中科技大學"
ISBN = str(9789574672028)
df_lst = []
url = "https://ntit.ent.sirsidynix.net/client/zh_TW/NUTC/search/results?qu=" + ISBN + "&te=ILS"


driver.get(url)
time.sleep(3)
into_first = driver.find_element_by_id("detailLink0").click() #進第一個版本
time.sleep(3)
for i in range(0, 6):
    booktype = driver.find_element_by_class_name("detailItemsTable_ITYPE").text
    if booktype == "電子資源": #不爬是電子資源的
        pass
    else:
        try:
            html_text = driver.page_source
            dfs = pd.read_html(html_text, encoding="utf-8")
            df_ntit = dfs[4]

            df_lst.append(df_ntit)
            next = driver.find_element_by_class_name("nextArrowRight").click() #按去下一個版本
        except:
            pass

print(df_lst)

[]
